<a href="https://colab.research.google.com/github/jiwoo50/dataSci/blob/main/datascience_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**데이터 과학 HW2 김지우**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/dataSci"
!dir

/content/drive/MyDrive/dataSci
cat.jpg  datascience_HW1.ipynb


In [5]:
import torch

# 1. Take several pictures of red, blue, and green items with your phone or other digital camera (or download some from the internet, if a camera isn’t available). 

**a. Load each image, and convert it to a tensor**

In [7]:
import imageio

img_arr = imageio.imread('./cat.jpg')
img_arr.shape

<ipython-input-7-9e761c11c93a>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('./cat.jpg')


(463, 703, 3)

In [37]:
img = torch.from_numpy(img_arr)
img.shape

torch.Size([463, 703, 3])

**b. For each image tensor, use the .mean() method to get a sense of how bright the image is**

In [42]:
torch.mean(img,dtype=torch.float32).shape

torch.Size([])

In [43]:
torch.mean(img,dtype=torch.float32)

tensor(101.1128)

**c. Take the mean of each channel of your images. Can you identify the red, green, and blue items from only the channel averages?**


red channel

In [41]:
torch.mean(img[:,:,0],dtype=torch.float32)

tensor(105.4064)

green channel

In [44]:
torch.mean(img[:,:,1],dtype=torch.float32)

tensor(129.3312)

blue channel

In [45]:
torch.mean(img[:,:,2],dtype=torch.float32)

tensor(68.6008)

# 1. Redefine the model to be w2 * t_u ** 2 + w1 * t_u + b. 

In [46]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [47]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [65]:
def model(t_u, w1,w2, b):
    return (w2 * t_u)**2 + w1 * t_u + b 

In [66]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [70]:
w1 = torch.ones(())
w2 = torch.ones(())
b = torch.zeros(())

t_p = model(t_u, w1,w2, b)
t_p

tensor([1310.1901, 3180.7100, 3445.4399, 6789.5103, 3225.9900, 2440.1101,
        1183.1101,  497.0399, 2390.9600, 3708.5601, 4746.9600])

In [71]:
loss = loss_fn(t_p, t_c)
loss

tensor(11709471.)

In [105]:
delta = 0.1

loss_rate_of_change_w = \
    (loss_fn(model(t_u, w1 + delta,w2 + delta, b), t_c) - 
     loss_fn(model(t_u, w1 - delta,w2 - delta, b), t_c)) / (2.0 * delta)

#loss_rate_of_change_w = loss_rate_of_change_w1+loss_rate_of_change_w2

In [75]:
learning_rate = 1e-2

w1 = w1 - learning_rate * loss_rate_of_change_w
w2 = w2 - learning_rate * loss_rate_of_change_w

In [76]:
loss_rate_of_change_b = \
    (loss_fn(model(t_u, w1, w2, b + delta), t_c) - 
     loss_fn(model(t_u, w1, w2, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [77]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)  # <1>
    return dsq_diffs

In [87]:
def dmodel_dw(t_u, w1,w2, b):
    return t_u

In [88]:
def dmodel_db(t_u, w1,w2, b):
    return 1.0

In [95]:
def grad_fn(t_u, t_c, t_p, w1,w2, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u,  w1,w2, b)
    dloss_db = dloss_dtp * dmodel_db(t_u,  w1,w2, b)
    return torch.stack([dloss_dw.sum(), dloss_dw.sum(),dloss_db.sum()])  # <1>

In [96]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w1,w2, b = params

        t_p = model(t_u, w1,w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1,w2, b)  # <2>

        params = params - learning_rate * grad

        print('Epoch %d, Loss %f' % (epoch, float(loss))) # <3>
            
    return params

In [97]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c,
                  print_params=True):
    for epoch in range(1, n_epochs + 1):
        w1,w2, b = params

        t_p = model(t_u, w1,w2, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w1,w2, b)  # <2>

        params = params - learning_rate * grad

        if epoch in {1, 2, 3, 10, 11, 99, 100, 4000, 5000}:  # <3>
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            if print_params:
                print('    Params:', params)
                print('    Grad:  ', grad)
        if epoch in {4, 12, 101}:
            print('...')

        if not torch.isfinite(loss).all():
            break  # <3>
            
    return params

In [108]:
training_loop(
    n_epochs = 100, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_u, 
    t_c = t_c)

Epoch 1, Loss 11709471.000000
    Params: tensor([-3617.1025, -3617.1025,   -59.6420])
    Grad:   tensor([361810.2500, 361810.2500,   5964.1958])
Epoch 2, Loss 1958411549322801840128.000000
    Params: tensor([-4.6746e+10, -4.6746e+10, -7.6951e+08])
    Grad:   tensor([4.6746e+12, 4.6746e+12, 7.6951e+10])
Epoch 3, Loss inf
    Params: tensor([-7.8075e+24, -7.8075e+24, -1.2852e+23])
    Grad:   tensor([7.8075e+26, 7.8075e+26, 1.2852e+25])


tensor([-7.8075e+24, -7.8075e+24, -1.2852e+23])

In [102]:
training_loop(
    n_epochs = 100, 
    learning_rate = 1e-4, 
    params = torch.tensor([1.0,1.0, 0.0]), 
    t_u = t_u, 
    t_c = t_c)

Epoch 1, Loss 11709471.000000
    Params: tensor([-35.1810, -35.1810,  -0.5964])
    Grad:   tensor([361810.2500, 361810.2500,   5964.1958])
Epoch 2, Loss 17510957056000.000000
    Params: tensor([-44236.6523, -44236.6523,   -728.1976])
    Grad:   tensor([4.4201e+08, 4.4201e+08, 7.2760e+06])
Epoch 3, Loss 43812013299240165403787264.000000
    Params: tensor([-6.9918e+10, -6.9918e+10, -1.1510e+09])
    Grad:   tensor([6.9918e+14, 6.9918e+14, 1.1510e+13])
...


tensor([-1.7466e+23, -1.7466e+23, -2.8752e+21])

**a. What parts of the training loop, and so on, need to change to accommodate this redefinition?**

a. 교육 루프에서 모델과 loss함수, 파라미터에 가중치를 추가해야한다


**b. What parts are agnostic to swapping out the model?**

b. 모델을 교체할 때 모델의 입력부분에 영향을 받는다.

**c. Is the resulting loss higher or lower after training?**



c. 손실은 더 높아졌다.







**d. Is the actual result better or worse?**


d. 실제 결과도 더 안좋아졌다.

# The third-hardest problem in physics is finding a proper wine to celebrate discoveries. Load the wine data from chapter 4, and create a new model with the appropriate number of input parameters. 

a How long does it take to train compared to the temperature data we have been using? 

b Can you explain what factors contribute to the training times? 

c Can you get the loss to decrease while training on this dataset? 

d How would you go about graphing this dataset?

In [117]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [110]:
import csv
wine_path = "./winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [111]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [112]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [113]:
data = wineq[:, :-1] # <1>
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [114]:
target = wineq[:, -1] # <2>
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [115]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data, target, test_size = 0.2)

In [116]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape 

(torch.Size([3918, 11]),
 torch.Size([980, 11]),
 torch.Size([3918]),
 torch.Size([980]))

In [146]:
linear_model = nn.Linear(11, 1) # <1>
linear_model(train_x).shape

torch.Size([3918, 1])

In [147]:
linear_model = nn.Linear(11, 1) # <1>
optimizer = optim.SGD(
    linear_model.parameters(), # <2>
    lr=1e-2)

In [148]:
linear_model.parameters()

<generator object Module.parameters at 0x7fe2d89806d0>

In [149]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.04,  0.29, -0.28,  0.21,  0.15, -0.25, -0.09,  0.23,  0.04,
          -0.24,  0.13]], requires_grad=True),
 Parameter containing:
 tensor([0.05], requires_grad=True)]

In [150]:
linear_model(train_x)

tensor([[-15.29],
        [-16.06],
        ...,
        [-10.99],
        [-28.65]], grad_fn=<AddmmBackward0>)

In [162]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [152]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [166]:
linear_model = nn.Linear(11, 1) # <1>
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 100, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = loss_fn,
    t_u_train = train_x,
    t_u_val = test_x, 
    t_c_train = train_y,
    t_c_val = test_y)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 13.4783, Validation loss 12.8206
Epoch 10, Training loss inf, Validation loss inf
Epoch 20, Training loss nan, Validation loss nan
Epoch 30, Training loss nan, Validation loss nan
Epoch 40, Training loss nan, Validation loss nan
Epoch 50, Training loss nan, Validation loss nan
Epoch 60, Training loss nan, Validation loss nan
Epoch 70, Training loss nan, Validation loss nan
Epoch 80, Training loss nan, Validation loss nan
Epoch 90, Training loss nan, Validation loss nan
Epoch 100, Training loss nan, Validation loss nan

Parameter containing:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       requires_grad=True)
Parameter containing:
tensor([nan], requires_grad=True)


계속 loss가 발산해서 제대로된 비교를 할 수 없었다.